In [1]:
import collections
import sys
sys.path.append('..')
from d2l import torch as d2l
import re

In [2]:
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt', '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('^A-Za-z', '', line).strip().lower() for line in lines]
lines = read_time_machine()
print(f'# text lines: {len(lines)}')
print(lines[0])
print(lines[10])

# text lines: 3221
the time machine, by h. g. wells [1898]
twinkled, and his usually pale face was flushed and animated. the


In [3]:
def tokenize(lines, token='word'):
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print("ERROR: unknown token type:" + token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine,', 'by', 'h.', 'g.', 'wells', '[1898]']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', '(for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him)']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us.', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled,', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated.', 'the']


In [4]:
class Vocab:
    def __init__(self, tokens=None, min_freq=0, reserved_tokens = None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        counter = count_corpus(tokens)
        self.token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)
        self.unk, uniq_tokens = 0, ['<unk>'] + reserved_tokens
        uniq_tokens +=  [token for token, freq in self.token_freqs if freq >= min_freq and token not in uniq_tokens]
        self.idx_to_token, self.token_to_idx = [], dict()
        for i, token in enumerate(uniq_tokens):
            self.idx_to_token.append(token)
            self.token_to_idx[token] = i
    
    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]
    
    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.to_tokens(indice) for indice in indices]

def count_corpus(tokens):
    if len(tokens) == 0 or isinstance(tokens[0], list):
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

In [5]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('and', 2), ('i', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('my', 9)]


In [6]:
for i in [0, 10]:
    print("words:", tokens[i])
    print("indices", vocab[tokens[i]])

words: ['the', 'time', 'machine,', 'by', 'h.', 'g.', 'wells', '[1898]']
indices [1, 18, 197, 37, 2413, 2414, 755, 2415]
words: ['twinkled,', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated.', 'the']
indices [2419, 2, 22, 996, 442, 154, 7, 1410, 2, 1411, 1]


In [7]:
def load_corpus_time_machine(max_tokens=-1):
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
print(corpus[:5], len(corpus), vocab.token_to_idx, len(vocab))


[3, 9, 2, 1, 3] 175758 {'<unk>': 0, ' ': 1, 'e': 2, 't': 3, 'a': 4, 'i': 5, 'n': 6, 'o': 7, 's': 8, 'h': 9, 'r': 10, 'd': 11, 'l': 12, 'm': 13, 'u': 14, 'c': 15, 'f': 16, 'w': 17, 'g': 18, 'y': 19, 'p': 20, ',': 21, 'b': 22, '.': 23, 'v': 24, 'k': 25, "'": 26, '-': 27, 'x': 28, 'z': 29, ';': 30, 'j': 31, '?': 32, 'q': 33, '!': 34, '_': 35, '"': 36, ':': 37, '(': 38, ')': 39, '[': 40, '8': 41, ']': 42, '1': 43, '9': 44} 45
